In [1]:
import torch, numpy as np, torch_geometric, random

In [2]:
device = torch.device("cuda")

In [4]:
class SEAL(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.gcn1 = torch_geometric.nn.GCNConv(16, 32)
        self.gcn2 = torch_geometric.nn.GCNConv(32, 32)
        self.gcn3 = torch_geometric.nn.GCNConv(32, 32)
        self.gcn4 = torch_geometric.nn.GCNConv(32, 1)
        self.global_pool = torch_geometric.nn.aggr.SortAggregation(k=30)
        self.conv1 = torch.nn.Conv1d(1, 16, 97, 97)
        self.conv2 = torch.nn.Conv1d(16, 32, 5, 1)
        self.maxpool = torch.nn.MaxPool1d(2, 2)
        self.linear1 = torch.nn.Linear(352, 128)
        self.linear2 = torch.nn.Linear(128, 1)
        self.dropout = torch.nn.Dropout(0.5)
    def forward(self, mat, edge_index):
        h1 = self.gcn1(mat, edge_index).tanh()
        h2 = self.gcn2(h1, edge_index).tanh()
        h3 = self.gcn3(h2, edge_index).tanh()
        h4 = self.gcn4(h3, edge_index).tanh()
        h = torch.cat([h1, h2, h3, h4], dim=-1)
        h = self.global_pool(h)
        h = h.view(h.size(0), 1, h.size(-1))
        h = self.conv1(h).relu()
        h = self.maxpool(h)
        h = self.conv2(h).relu()
        h = h.view(h.size(0), -1)
        h = self.linear1(h).relu()
        h = self.dropout(h)
        h = self.linear2(h).sigmoid()
        return h

In [5]:
model = SEAL()
model.train(True)
model.to(device)
optimizer = torch.optim.Adam(params=model.parameters(), lr = 0.0005623324269475477)
criterion = torch.nn.BCELoss()

In [5]:
def train_one_epoch(k):
    running_loss = 0.
    last_loss = 0.

    # Here, we use enumerate(training_loader) instead of
    # iter(training_loader) so that we can track the batch
    # index and do some intra-epoch reporting
    pos_batch_dat = np.load(f'pos_dat/pos_batch_{k}.npz', allow_pickle=True)
    neg_batch_dat = np.load(f'neg_dat/neg_batch_{k}.npz', allow_pickle=True)
    pos_table = pos_batch_dat.f.arr_0
    neg_table = neg_batch_dat.f.arr_0
    lab_t = [j for j in range(len(pos_table)+len(neg_table)-1)]
    random.shuffle(lab_t)
    print(f"Batch {k} in progress")
    ind = 0
    for i in lab_t:
        label = None
        g = None
        x = None
        if i < len(pos_table):
            label = torch.tensor([1.], dtype=torch.float32).to(device)
            g = pos_table[i][2]
            x = pos_table[i][3]
        else:
            label = torch.tensor([0.], dtype=torch.float32).to(device)
            g = neg_table[i - len(pos_table)][2]
            x = neg_table[i - len(pos_table)][3]
        # Zero your gradients for every batch!
        optimizer.zero_grad()

        # Make predictions for this batch
        inp = torch.tensor(g, dtype=torch.float32).to_sparse_coo().abs().coalesce().to(device)
        outputs = model(torch.tensor(x).to(torch.float32).to(device), inp)

        # Compute the loss and its gradients
        loss = criterion(outputs.view(1), label)
        loss.backward()

        # Adjust learning weights
        optimizer.step()

        # Gather data and report
        ind +=1
        running_loss += loss.item()
        if ind == 197:
            last_loss = running_loss # loss per batch
            print(f'Output: {outputs}, label: {label}')
            print('  batch {} loss: {}'.format(i + 1, last_loss))
            running_loss = 0.
    return last_loss

In [6]:
for epoch in range(8):
    print(f"Epoch {epoch} in progress")
    for i in range(1, 31):
        train_one_epoch(i)

Epoch 0 in progress
Batch 1 in progress


/home/dp1200@unt.ad.unt.edu/.venv/lib64/python3.9/site-packages/torch_geometric/utils/spmm.py:68: UserWarning: Converting sparse tensor to CSR format for more efficient processing. Consider converting your sparse tensor to CSR format beforehand to avoid repeated conversion (got 'torch.sparse_coo')
  warnings.warn(f"Converting sparse tensor to CSR format for more "
/home/dp1200@unt.ad.unt.edu/.venv/lib64/python3.9/site-packages/torch_geometric/utils/spmm.py:72: UserWarning: Sparse CSR tensor support is in beta state. If you miss a functionality in the sparse tensor support, please submit a feature request to https://github.com/pytorch/pytorch/issues. (Triggered internally at ../aten/src/ATen/SparseCsrTensorImpl.cpp:53.)
  src = src.to_sparse_csr()


Output: tensor([[0.4932]], device='cuda:0', grad_fn=<SigmoidBackward0>), label: tensor([1.], device='cuda:0')
  batch 37 loss: 136.99758678674698
Batch 2 in progress
Output: tensor([[0.4532]], device='cuda:0', grad_fn=<SigmoidBackward0>), label: tensor([1.], device='cuda:0')
  batch 98 loss: 136.64399671554565
Batch 3 in progress
Output: tensor([[0.1536]], device='cuda:0', grad_fn=<SigmoidBackward0>), label: tensor([0.], device='cuda:0')
  batch 111 loss: 67.745118774107
Batch 4 in progress
Output: tensor([[0.0349]], device='cuda:0', grad_fn=<SigmoidBackward0>), label: tensor([0.], device='cuda:0')
  batch 186 loss: 34.331704487053855
Batch 5 in progress
Output: tensor([[0.9961]], device='cuda:0', grad_fn=<SigmoidBackward0>), label: tensor([1.], device='cuda:0')
  batch 62 loss: 62.07340895408561
Batch 6 in progress
Output: tensor([[0.1147]], device='cuda:0', grad_fn=<SigmoidBackward0>), label: tensor([0.], device='cuda:0')
  batch 128 loss: 73.28360286761125
Batch 7 in progress
Output

In [8]:
expecteds = []
actuals = []
probs = []

In [9]:
torch.no_grad()
model.train(False)
for k in range(31,40):
    pos_batch_dat = np.load(f'pos_dat/pos_batch_{k}.npz', allow_pickle=True)
    neg_batch_dat = np.load(f'neg_dat/neg_batch_{k}.npz', allow_pickle=True)
    pos_table = pos_batch_dat.f.arr_0
    neg_table = neg_batch_dat.f.arr_0
    lab_t = [j for j in range(len(pos_table)+len(neg_table)-1)]
    random.shuffle(lab_t)
    print(f"Batch {k} in progress")
    ind = 0
    outs = []
    temp_l = []
    for i in lab_t:
        label = None
        g = None
        x = None
        e = None
        if i < len(pos_table):
            temp_l.append(1)
            g = pos_table[i][2]
            x = pos_table[i][3]
        else:
            temp_l.append(0)
            g = neg_table[i - len(pos_table)][2]
            x = neg_table[i - len(pos_table)][3]

        # Make predictions for this batch
        inp = torch.tensor(g, dtype=torch.float32).to_sparse_coo().abs().coalesce().to(device)
        outputs = model( torch.tensor(x).to(torch.float32).to(device), inp)
        print(outputs)
        probs.append(float(outputs.view(1)[0]))
        o = 0
        if outputs.view(1)[0] > 0.5:
            o = 1
        outs.append(o)
        torch.cuda.empty_cache()
    expecteds.extend(temp_l)
    actuals.extend(outs)
    torch.cuda.empty_cache()



Batch 31 in progress
tensor([[0.4981]], device='cuda:0', grad_fn=<SigmoidBackward0>)
tensor([[0.4981]], device='cuda:0', grad_fn=<SigmoidBackward0>)
tensor([[0.4981]], device='cuda:0', grad_fn=<SigmoidBackward0>)
tensor([[0.5130]], device='cuda:0', grad_fn=<SigmoidBackward0>)
tensor([[0.4981]], device='cuda:0', grad_fn=<SigmoidBackward0>)
tensor([[0.4982]], device='cuda:0', grad_fn=<SigmoidBackward0>)
tensor([[0.4981]], device='cuda:0', grad_fn=<SigmoidBackward0>)
tensor([[0.4981]], device='cuda:0', grad_fn=<SigmoidBackward0>)
tensor([[0.4464]], device='cuda:0', grad_fn=<SigmoidBackward0>)
tensor([[0.4981]], device='cuda:0', grad_fn=<SigmoidBackward0>)
tensor([[0.2741]], device='cuda:0', grad_fn=<SigmoidBackward0>)
tensor([[0.4981]], device='cuda:0', grad_fn=<SigmoidBackward0>)
tensor([[0.4981]], device='cuda:0', grad_fn=<SigmoidBackward0>)
tensor([[0.4981]], device='cuda:0', grad_fn=<SigmoidBackward0>)
tensor([[0.4981]], device='cuda:0', grad_fn=<SigmoidBackward0>)
tensor([[0.5129]], 

In [10]:
sum = 0.
for l in range(len(expecteds)):
    if expecteds[l] == actuals[l]:
        sum += 1
sum/len(actuals)

0.5352509870276367

In [18]:
from sklearn.metrics import roc_curve, auc

In [29]:
fpr, tpr, thr = roc_curve(np.array(expecteds).astype(np.int64), np.array(probs).astype(np.float64), pos_label=1)

In [30]:
auc(fpr, tpr)

0.7847287437234527

In [28]:
len(probs)

1773

In [11]:
torch.save(model.state_dict(), './DGCNN')

In [6]:
model.load_state_dict(torch.load('./DGCNN'))

<All keys matched successfully>